In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:95% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:15pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:15pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
"""))

# <span style="color:red"> Step.3_추출 데이터 기본정제 </span>

In [2]:
import os
import re

def basic_text_cleaning(text):
    """기본적인 텍스트 정제"""
    
    print("기본 정제 시작...")
    
    # 1. 과도한 줄바꿈 정리 (3개 이상 → 2개)
    text = re.sub(r'\n\s*\n\s*\n+', '\n\n', text)
    print("✓ 과도한 줄바꿈 정리 완료")
    
    # 2. 각 줄의 앞뒤 공백 제거 및 중간 공백 정리
    lines = text.split('\n')
    cleaned_lines = []
    
    for line in lines:
        # 앞뒤 공백 제거
        line = line.strip()
        
        # 과도한 중간 공백 정리 (2개 이상 공백을 1개로)
        line = re.sub(r'\s{2,}', ' ', line)
        
        # 탭 문자를 공백으로 변환
        line = line.replace('\t', ' ')
        
        cleaned_lines.append(line)
    
    print("✓ 공백 정리 완료")
    
    # 3. 연속된 빈 줄 정리
    final_lines = []
    prev_empty = False
    
    for line in cleaned_lines:
        if line == "":
            if not prev_empty:
                final_lines.append(line)
            prev_empty = True
        else:
            final_lines.append(line)
            prev_empty = False
    
    print("✓ 빈 줄 정리 완료")
    
    return '\n'.join(final_lines)

def remove_repetitive_headers(text):
    """반복되는 헤더/푸터/제목 제거"""
    
    print("중복 콘텐츠 제거 시작...")
    
    lines = text.split('\n')
    
    # 페이지 번호만 있는 줄 제거
    lines = [line for line in lines if not re.match(r'^\s*\d+\s*$', line)]
    print("✓ 페이지 번호 제거 완료")
    
    # 자주 반복되는 줄 찾기 (5자 이상의 의미있는 줄만)
    line_counts = {}
    for line in lines:
        if line.strip() and len(line.strip()) > 5:
            clean_line = line.strip()
            line_counts[clean_line] = line_counts.get(clean_line, 0) + 1
    
    # 3번 이상 반복되는 줄들을 헤더/푸터로 간주
    repetitive_lines = set()
    for line, count in line_counts.items():
        if count >= 3:
            # 단, 중요한 법령이나 조문은 제외
            if not any(keyword in line for keyword in ['조', '항', '과실비율', '도로교통법']):
                repetitive_lines.add(line)
    
    print(f"✓ {len(repetitive_lines)}개의 반복 줄 발견")
    
    # 반복 줄 제거
    cleaned_lines = []
    for line in lines:
        if line.strip() not in repetitive_lines:
            cleaned_lines.append(line)
    
    print("✓ 중복 콘텐츠 제거 완료")
    
    return '\n'.join(cleaned_lines)

def normalize_headers(text):
    """제목 구조 정규화"""
    
    print("제목 구조 정규화 시작...")
    
    lines = text.split('\n')
    normalized_lines = []
    
    for line in lines:
        original_line = line
        line = line.strip()
        
        if not line:
            normalized_lines.append('')
            continue
        
        # 이미 마크다운 헤더인 경우 패스
        if line.startswith('#'):
            normalized_lines.append(original_line)
            continue
        
        # 1. 페이지 제목 (# 페이지 X)
        if line.startswith('페이지 ') and re.match(r'^페이지\s+\d+', line):
            normalized_lines.append(f"# {line}")
            continue
        
        # 2. 숫자로 시작하는 대제목 (1., 2., 1), 2) 등)
        if re.match(r'^[\d]+[\.\)]\s+[가-힣]', line):
            normalized_lines.append(f"## {line}")
            continue
        
        # 3. 조문 제목 (제1조, 제2조 등)
        if re.match(r'^제\s*\d+\s*조', line):
            normalized_lines.append(f"### {line}")
            continue
        
        # 4. 항목 제목 (가., 나., 다. 등)
        if re.match(r'^[가-힣][\.\)]\s+', line):
            normalized_lines.append(f"#### {line}")
            continue
        
        # 5. 소항목 (①, ②, ③ 등)
        if re.match(r'^[①②③④⑤⑥⑦⑧⑨⑩]\s+', line):
            normalized_lines.append(f"##### {line}")
            continue
        
        # 6. 중요 키워드가 포함된 줄 강조
        important_keywords = ['과실비율', '기본과실', '도로교통법', '인정기준']
        if any(keyword in line for keyword in important_keywords) and len(line) < 100:
            normalized_lines.append(f"**{line}**")
            continue
        
        # 일반 텍스트
        normalized_lines.append(line)
    
    print("✓ 제목 구조 정규화 완료")
    
    return '\n'.join(normalized_lines)

def remove_special_characters(text):
    """불필요한 특수문자 정리"""
    
    print("특수문자 정리 시작...")
    
    # PDF 변환 시 생기는 이상한 문자들 제거
    text = re.sub(r'[^\w\s가-힣ㄱ-ㅎㅏ-ㅣ\.\,\:\;\(\)\[\]\-\+\=\%\#\*\|\~\`\"\'\?\!]', ' ', text)
    
    # 연속된 특수문자 정리
    text = re.sub(r'[\.\,\:\;]{2,}', '.', text)
    
    print("✓ 특수문자 정리 완료")
    
    return text

def clean_markdown_file(file_path):
    """마크다운 파일 정제"""
    
    if not os.path.exists(file_path):
        print(f"❌ 파일을 찾을 수 없습니다: {file_path}")
        return None
    
    print(f"\n{'='*60}")
    print(f"파일 정제 시작: {os.path.basename(file_path)}")
    print(f"{'='*60}")
    
    # 파일 읽기
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    original_size = len(content)
    original_lines = len(content.split('\n'))
    
    print(f"원본 파일 정보:")
    print(f"  - 문자 수: {original_size:,}")
    print(f"  - 줄 수: {original_lines:,}")
    
    # 단계별 정제
    content = basic_text_cleaning(content)
    content = remove_repetitive_headers(content)
    content = normalize_headers(content)
    content = remove_special_characters(content)
    
    # 최종 정리
    content = basic_text_cleaning(content)  # 한번 더 기본 정제
    
    final_size = len(content)
    final_lines = len(content.split('\n'))
    
    print(f"\n정제 결과:")
    print(f"  - 최종 문자 수: {final_size:,} ({original_size-final_size:+,})")
    print(f"  - 최종 줄 수: {final_lines:,} ({original_lines-final_lines:+,})")
    print(f"  - 압축률: {(1-final_size/original_size)*100:.1f}%")
    
    # 정제된 파일 저장
    base_name = os.path.splitext(os.path.basename(file_path))[0]
    output_path = os.path.join(os.path.dirname(file_path), f"{base_name}_기본정제.md")
    
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(content)
    
    print(f"✅ 정제 완료: {output_path}")
    
    return content, output_path

def preview_cleaned_content(content, num_lines=30):
    """정제된 내용 미리보기"""
    
    lines = content.split('\n')
    print(f"\n📋 정제 결과 미리보기 (처음 {num_lines}줄):")
    print("-" * 50)
    
    for i, line in enumerate(lines[:num_lines]):
        if line.strip():
            print(f"{i+1:3d}: {line}")
        else:
            print(f"{i+1:3d}: ")
    
    if len(lines) > num_lines:
        print(f"... (총 {len(lines):,}줄 중 {num_lines}줄만 표시)")

# 메인 실행
def main():
    # 파일 경로
    file1_path = r"C:\project\2stProject_jun\jun\과실비율PDF\231107_과실비율인정기준_온라인용.md"
    file2_path = r"C:\project\2stProject_jun\jun\과실비율PDF\(최종)과실비율심의사례_(54MB).md"
    
    # 두 파일 정제
    results = []
    
    for file_path in [file1_path, file2_path]:
        result = clean_markdown_file(file_path)
        if result:
            content, output_path = result
            results.append({
                'original_path': file_path,
                'output_path': output_path,
                'content': content,
                'filename': os.path.basename(output_path)
            })
            
            # 미리보기
            preview_cleaned_content(content)
    
    # 전체 결과 요약
    print(f"\n{'='*60}")
    print("전체 정제 결과 요약")
    print(f"{'='*60}")
    
    total_chars = 0
    for i, result in enumerate(results):
        char_count = len(result['content'])
        total_chars += char_count
        print(f"{i+1}. {result['filename']}")
        print(f"   - 문자 수: {char_count:,}")
        print(f"   - 저장 위치: {result['output_path']}")
    
    print(f"\n총 문자 수: {total_chars:,}")
    print(f"처리된 파일: {len(results)}개")
    
    print(f"\n다음 단계:")
    print("1. 정제 결과 확인")
    print("2. 추가 고급 정제 필요성 판단")
    print("3. 만족스러우면 통합 작업 진행")
    
    return results

# 실행
if __name__ == "__main__":
    results = main()


파일 정제 시작: 231107_과실비율인정기준_온라인용.md
원본 파일 정보:
  - 문자 수: 561,331
  - 줄 수: 26,109
기본 정제 시작...
✓ 과도한 줄바꿈 정리 완료
✓ 공백 정리 완료
✓ 빈 줄 정리 완료
중복 콘텐츠 제거 시작...
✓ 페이지 번호 제거 완료
✓ 543개의 반복 줄 발견
✓ 중복 콘텐츠 제거 완료
제목 구조 정규화 시작...
✓ 제목 구조 정규화 완료
특수문자 정리 시작...
✓ 특수문자 정리 완료
기본 정제 시작...
✓ 과도한 줄바꿈 정리 완료
✓ 공백 정리 완료
✓ 빈 줄 정리 완료

정제 결과:
  - 최종 문자 수: 383,892 (+177,439)
  - 최종 줄 수: 17,653 (+8,456)
  - 압축률: 31.6%
✅ 정제 완료: C:\project\2stProject_jun\jun\과실비율PDF\231107_과실비율인정기준_온라인용_기본정제.md

📋 정제 결과 미리보기 (처음 30줄):
--------------------------------------------------
  1: # 페이지 1
  2: 
  3: 2023.6.
  4: 자동차사고
  5: **과실비율**
  6: **인정기준**
  7: 
  8: # 페이지 2
  9: 
 10: **자동차사고 과실비율 인정기준**
 11: 발간사.006
 12: 제1편 개정경과.009
 13: 제2편 총 설.011
 14: ### 1. 과실비율 인정기준의 필요성.012
 15: ### 2. 과실과 과실상계.014
 16: (1) 과실의 의의.014
 17: (2) 피해자 과실상계의 의의.014
 18: (3) 피해자 과실상계의 법적 근거.015
 19: ### 3. 과실비율 인정기준의 기본원칙.016
 20: (1) 신뢰의 원칙 - 예견가능성.016
 21: ### 1) 자동차 대 자동차 .016
 22: ### 2) 자동차 대 보행자.018
 23: (2) 과실상계의 기본조건 - 인과관계.018
 24: ### 4. 과실비율 인정기준

# 2. 추가보완 기본정제

In [4]:
import re
import os

def extract_case_data_from_markdown(content):
    """마크다운에서 사례 데이터 추출"""
    
    cases = []
    lines = content.split('\n')
    current_case = None
    
    for i, line in enumerate(lines):
        line = line.strip()
        if not line:
            continue
        
        # 사례 코드 패턴 감지 (보1, 차1, 교1 등)
        case_pattern = r'^([가-힣]+)(\d+)\s*[:：]\s*(.+)'
        if re.match(case_pattern, line):
            # 이전 사례 저장
            if current_case:
                cases.append(current_case)
            
            match = re.match(case_pattern, line)
            case_type = match.group(1)
            case_number = match.group(2)
            case_description = match.group(3)
            
            current_case = {
                'code': f"{case_type}{case_number}",
                'type': case_type,
                'number': case_number,
                'title': case_description,
                'participants': {},
                'basic_fault_ratio': None,
                'adjustments': [],
                'non_applicable': []
            }
            continue
        
        if not current_case:
            continue
        
        # 참여자 정보 ((보), (차) 등)
        participant_pattern = r'^\(([가-힣]+)\)\s*(.+)'
        if re.match(participant_pattern, line):
            match = re.match(participant_pattern, line)
            participant = match.group(1)
            description = match.group(2)
            current_case['participants'][participant] = description
            continue
        
        # 기본 과실비율
        fault_patterns = [
            r'([가-힣]+)\s*기본\s*과실비율?\s*[:：]\s*(\d+)',
            r'기본\s*과실\s*[:：]\s*(.+)',
            r'과실비율?\s*[:：]\s*(\d+)\s*[:：]\s*(\d+)'
        ]
        
        for pattern in fault_patterns:
            if re.search(pattern, line):
                current_case['basic_fault_ratio'] = line
                break
        
        # 조정 요소 (①, ②, +5, -10 등)
        # 숫자나 기호로 시작하는 조정 항목들
        adjustment_pattern = r'([①②③④⑤⑥⑦⑧⑨⑩\d]+\.?)\s*(.+?)(?:\s+([+\-]\d+)|(\+\d+|\-\d+))'
        if re.search(adjustment_pattern, line):
            match = re.search(adjustment_pattern, line)
            number = match.group(1)
            description = match.group(2).strip()
            value = match.group(3) or match.group(4)
            
            current_case['adjustments'].append({
                'number': number,
                'description': description,
                'value': value
            })
            continue
        
        # 비적용 항목
        if '비적용' in line:
            description = line.replace('비적용', '').strip()
            current_case['non_applicable'].append(description)
            continue
        
        # 단순 조정 항목 (들여쓰기로 된 세부 항목들)
        if line.startswith('    ') or line.startswith('\t'):
            clean_line = line.strip()
            # +숫자 또는 -숫자가 포함된 경우
            value_pattern = r'(.+?)\s*([+\-]\d+)(?:\s|$)'
            match = re.search(value_pattern, clean_line)
            if match:
                description = match.group(1).strip()
                value = match.group(2)
                current_case['adjustments'].append({
                    'number': '',
                    'description': description,
                    'value': value
                })
    
    # 마지막 사례 저장
    if current_case:
        cases.append(current_case)
    
    return cases

def format_case_as_table_markdown(case):
    """사례를 표 형태 마크다운으로 변환"""
    
    markdown = []
    
    # 제목
    markdown.append(f"## {case['code']}: {case['title']}")
    markdown.append("")
    
    # 참여자 정보
    if case['participants']:
        for participant, description in case['participants'].items():
            markdown.append(f"**({participant})** {description}")
        markdown.append("")
    
    # 기본 과실비율
    if case['basic_fault_ratio']:
        markdown.append(f"**기본 과실비율**: {case['basic_fault_ratio']}")
        markdown.append("")
    
    # 조정 요소 표
    if case['adjustments'] or case['non_applicable']:
        markdown.append("### 과실비율 조정요소")
        markdown.append("")
        markdown.append("| 항목 | 조정 내용 | 조정값 |")
        markdown.append("|------|-----------|--------|")
        
        # 조정 요소
        for adj in case['adjustments']:
            number = adj['number'] if adj['number'] else ''
            description = adj['description']
            value = adj['value'] if adj['value'] else ''
            markdown.append(f"| {number} | {description} | {value} |")
        
        # 비적용 항목
        for non_app in case['non_applicable']:
            markdown.append(f"| | {non_app} | 비적용 |")
        
        markdown.append("")
    
    markdown.append("---")
    markdown.append("")
    
    return '\n'.join(markdown)

def clean_and_restructure_markdown(content):
    """마크다운 내용을 깨끗하게 정제하고 표 구조로 재구성"""
    
    # 기본 텍스트 정제
    content = basic_text_cleaning(content)
    
    # 사례 데이터 추출
    cases = extract_case_data_from_markdown(content)
    
    # 표 형태로 재구성
    restructured_content = []
    
    # 문서 헤더
    restructured_content.append("# 자동차 사고 과실비율 인정기준")
    restructured_content.append("")
    restructured_content.append(f"총 {len(cases)}개 사례")
    restructured_content.append("")
    restructured_content.append("---")
    restructured_content.append("")
    
    # 각 사례를 표 형태로 변환
    for case in cases:
        case_markdown = format_case_as_table_markdown(case)
        restructured_content.append(case_markdown)
    
    return '\n'.join(restructured_content)

def basic_text_cleaning(text):
    """기본적인 텍스트 정제"""
    
    # 과도한 줄바꿈 정리
    text = re.sub(r'\n\s*\n\s*\n+', '\n\n', text)
    
    # 각 줄의 앞뒤 공백 제거
    lines = text.split('\n')
    cleaned_lines = []
    
    for line in lines:
        line = line.strip()
        # 과도한 중간 공백 정리
        line = re.sub(r'\s{2,}', ' ', line)
        # 탭을 공백으로 변환
        line = line.replace('\t', ' ')
        cleaned_lines.append(line)
    
    # 연속된 빈 줄 정리
    final_lines = []
    prev_empty = False
    
    for line in cleaned_lines:
        if line == "":
            if not prev_empty:
                final_lines.append(line)
            prev_empty = True
        else:
            final_lines.append(line)
            prev_empty = False
    
    return '\n'.join(final_lines)

def process_markdown_file(file_path):
    """마크다운 파일을 표 구조로 정제"""
    
    if not os.path.exists(file_path):
        print(f"❌ 파일을 찾을 수 없습니다: {file_path}")
        return None
    
    print(f"📄 파일 처리 시작: {os.path.basename(file_path)}")
    
    # 파일 읽기
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    original_size = len(content)
    original_lines = len(content.split('\n'))
    
    print(f"  📊 원본 정보: {original_size:,} 문자, {original_lines:,} 줄")
    
    # 정제 및 재구성
    restructured_content = clean_and_restructure_markdown(content)
    
    final_size = len(restructured_content)
    final_lines = len(restructured_content.split('\n'))
    
    print(f"  ✅ 정제 완료: {final_size:,} 문자, {final_lines:,} 줄")
    
    # 정제된 파일 저장
    base_name = os.path.splitext(os.path.basename(file_path))[0]
    output_path = os.path.join(os.path.dirname(file_path), f"{base_name}_표정제.md")
    
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(restructured_content)
    
    print(f"  💾 저장 완료: {output_path}")
    
    return restructured_content, output_path

def preview_restructured_content(content, num_lines=50):
    """정제된 내용 미리보기"""
    
    lines = content.split('\n')
    print(f"\n📋 정제 결과 미리보기 (처음 {num_lines}줄):")
    print("=" * 60)
    
    for i, line in enumerate(lines[:num_lines]):
        print(f"{i+1:3d}: {line}")
    
    if len(lines) > num_lines:
        print(f"... (총 {len(lines):,}줄 중 {num_lines}줄만 표시)")

def main():
    """메인 실행 함수"""
    
    # 파일 경로
    file_paths = [
        r"C:\project\2stProject_jun\jun\과실비율PDF\추출 마크다운\231107_과실비율인정기준_온라인용.md",
        r"C:\project\2stProject_jun\jun\과실비율PDF\추출 마크다운\(최종)과실비율심의사례_(54MB).md"
    ]
    
    results = []
    
    print("🚀 마크다운 표 구조 정제 시작")
    print("=" * 60)
    
    for file_path in file_paths:
        result = process_markdown_file(file_path)
        if result:
            content, output_path = result
            results.append({
                'original_path': file_path,
                'output_path': output_path,
                'content': content,
                'filename': os.path.basename(output_path)
            })
            
            # 미리보기
            preview_restructured_content(content)
            print("\n" + "="*60 + "\n")
    
    # 전체 결과 요약
    print("📊 전체 정제 결과 요약")
    print("=" * 60)
    
    total_chars = 0
    for i, result in enumerate(results):
        char_count = len(result['content'])
        total_chars += char_count
        print(f"{i+1}. {result['filename']}")
        print(f"   📄 문자 수: {char_count:,}")
        print(f"   💾 저장 위치: {result['output_path']}")
        print()
    
    print(f"📈 총 문자 수: {total_chars:,}")
    print(f"📁 처리된 파일: {len(results)}개")
    
    print("\n🎯 다음 단계:")
    print("1. 정제된 표 구조 확인")
    print("2. 필요시 추가 조정")
    print("3. DOCS 통합 변환 진행")
    
    return results

# 실행
if __name__ == "__main__":
    results = main()

🚀 마크다운 표 구조 정제 시작
📄 파일 처리 시작: 231107_과실비율인정기준_온라인용.md
  📊 원본 정보: 561,331 문자, 26,109 줄
  ✅ 정제 완료: 33 문자, 6 줄
  💾 저장 완료: C:\project\2stProject_jun\jun\과실비율PDF\추출 마크다운\231107_과실비율인정기준_온라인용_표정제.md

📋 정제 결과 미리보기 (처음 50줄):
  1: # 자동차 사고 과실비율 인정기준
  2: 
  3: 총 0개 사례
  4: 
  5: ---
  6: 


📄 파일 처리 시작: (최종)과실비율심의사례_(54MB).md
  📊 원본 정보: 445,586 문자, 19,855 줄
  ✅ 정제 완료: 33 문자, 6 줄
  💾 저장 완료: C:\project\2stProject_jun\jun\과실비율PDF\추출 마크다운\(최종)과실비율심의사례_(54MB)_표정제.md

📋 정제 결과 미리보기 (처음 50줄):
  1: # 자동차 사고 과실비율 인정기준
  2: 
  3: 총 0개 사례
  4: 
  5: ---
  6: 


📊 전체 정제 결과 요약
1. 231107_과실비율인정기준_온라인용_표정제.md
   📄 문자 수: 33
   💾 저장 위치: C:\project\2stProject_jun\jun\과실비율PDF\추출 마크다운\231107_과실비율인정기준_온라인용_표정제.md

2. (최종)과실비율심의사례_(54MB)_표정제.md
   📄 문자 수: 33
   💾 저장 위치: C:\project\2stProject_jun\jun\과실비율PDF\추출 마크다운\(최종)과실비율심의사례_(54MB)_표정제.md

📈 총 문자 수: 66
📁 처리된 파일: 2개

🎯 다음 단계:
1. 정제된 표 구조 확인
2. 필요시 추가 조정
3. DOCS 통합 변환 진행


# 3. 마크다운 표정제

In [6]:
import fitz  # PyMuPDF
import re
import os

def extract_and_structure_from_pdf(pdf_path):
    """PDF에서 바로 표 구조로 정제된 마크다운 생성"""
    
    if not os.path.exists(pdf_path):
        print(f"❌ 파일을 찾을 수 없습니다: {pdf_path}")
        return None
    
    print(f"📄 PDF 처리 시작: {os.path.basename(pdf_path)}")
    
    doc = fitz.open(pdf_path)
    cases = []
    raw_text = ""
    
    # 전체 텍스트 추출
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text = page.get_text()
        raw_text += text + "\n"
        print(f"  📄 페이지 {page_num + 1}/{len(doc)} 추출 완료")
    
    doc.close()
    
    print(f"  📊 총 추출 텍스트: {len(raw_text):,} 문자")
    
    # 텍스트에서 사례 추출
    cases = extract_cases_from_text(raw_text)
    
    print(f"  🔍 추출된 사례 수: {len(cases)}")
    
    # 표 형태 마크다운 생성
    structured_markdown = create_table_markdown(cases, os.path.basename(pdf_path))
    
    return structured_markdown, cases

def extract_cases_from_text(text):
    """텍스트에서 사례 데이터 추출"""
    
    cases = []
    lines = text.split('\n')
    current_case = None
    
    for i, line in enumerate(lines):
        line = line.strip()
        if not line:
            continue
        
        # 사례 코드 패턴 (보1, 차1, 교1 등)
        case_patterns = [
            r'^([보차교가나다라마바사아자차카타파하])\s*(\d+)\s*[:：]\s*(.+)',
            r'^([보차교가나다라마바사아자차카타파하])(\d+)\s+(.+)',
            r'^(\d+)\.\s*([보차교].+?)[:：]\s*(.+)'
        ]
        
        for pattern in case_patterns:
            match = re.match(pattern, line)
            if match:
                if current_case:
                    cases.append(current_case)
                
                if pattern == case_patterns[2]:  # 숫자로 시작하는 패턴
                    case_code = f"{match.group(2)[:1]}{match.group(1)}"
                    case_title = match.group(3)
                else:
                    case_code = f"{match.group(1)}{match.group(2)}"
                    case_title = match.group(3)
                
                current_case = {
                    'code': case_code,
                    'title': case_title,
                    'participants': {},
                    'basic_fault_ratio': None,
                    'fault_details': {},
                    'adjustments': [],
                    'special_conditions': []
                }
                break
        
        if not current_case:
            continue
        
        # 참여자 정보 ((보), (차) 등)
        participant_pattern = r'^\(([보차가나다라마바사아자차카타파하])\)\s*(.+)'
        if re.match(participant_pattern, line):
            match = re.match(participant_pattern, line)
            participant = match.group(1)
            description = match.group(2)
            current_case['participants'][participant] = description
            continue
        
        # 기본 과실비율 패턴들
        fault_patterns = [
            r'([보차가나다라마바사아자차카타파하])\s*(?:기본\s*)?과실비율?\s*[:：]\s*(\d+)',
            r'기본\s*과실\s*[:：]\s*([보차]\s*\d+%?\s*[:：]\s*[보차]\s*\d+%?)',
            r'과실비율?\s*[:：]\s*(\d+)\s*[:：]\s*(\d+)',
            r'([보차])\s*(\d+)%?\s*[:：]\s*([보차])\s*(\d+)%?'
        ]
        
        for pattern in fault_patterns:
            match = re.search(pattern, line)
            if match:
                if pattern == fault_patterns[0]:  # 단일 참여자
                    participant = match.group(1)
                    ratio = match.group(2)
                    current_case['fault_details'][participant] = ratio
                elif pattern == fault_patterns[3]:  # 두 참여자
                    p1, r1, p2, r2 = match.groups()
                    current_case['fault_details'][p1] = r1
                    current_case['fault_details'][p2] = r2
                else:
                    current_case['basic_fault_ratio'] = line
                break
        
        # 조정 요소들 (①, ②, +5, -10 등)
        adjustment_patterns = [
            r'([①②③④⑤⑥⑦⑧⑨⑩])\s*(.+?)\s*([+\-]\d+)',
            r'(.+?)\s*([+\-]\d+)(?:\s|$)',
            r'^[\s]*([가-힣\s,]+)\s*([+\-]\d+)$'
        ]
        
        for pattern in adjustment_patterns:
            match = re.search(pattern, line)
            if match:
                if pattern == adjustment_patterns[0]:
                    number = match.group(1)
                    description = match.group(2).strip()
                    value = match.group(3)
                else:
                    number = ''
                    description = match.group(1).strip()
                    value = match.group(2)
                
                # 중복 확인
                exists = any(adj['description'] == description for adj in current_case['adjustments'])
                if not exists and description:
                    current_case['adjustments'].append({
                        'number': number,
                        'description': description,
                        'value': value
                    })
                break
        
        # 비적용 항목
        if '비적용' in line:
            description = re.sub(r'비적용', '', line).strip()
            if description:
                current_case['special_conditions'].append({
                    'type': '비적용',
                    'description': description
                })
        
        # 도로교통법 참조
        law_match = re.search(r'도로교통법\s*제?\s*(\d+)\s*조', line)
        if law_match:
            law_ref = f"도로교통법 제{law_match.group(1)}조"
            if 'related_laws' not in current_case:
                current_case['related_laws'] = []
            if law_ref not in current_case['related_laws']:
                current_case['related_laws'].append(law_ref)
    
    # 마지막 사례 추가
    if current_case:
        cases.append(current_case)
    
    return cases

def create_table_markdown(cases, filename):
    """사례들을 표 형태 마크다운으로 변환"""
    
    markdown_lines = []
    
    # 문서 헤더
    doc_title = os.path.splitext(filename)[0]
    markdown_lines.append(f"# {doc_title}")
    markdown_lines.append("")
    markdown_lines.append(f"📊 **총 {len(cases)}개 사례**")
    markdown_lines.append("")
    markdown_lines.append("---")
    markdown_lines.append("")
    
    # 각 사례를 표 형태로 변환
    for case in cases:
        # 사례 제목
        markdown_lines.append(f"## {case['code']}: {case['title']}")
        markdown_lines.append("")
        
        # 참여자 정보
        if case['participants']:
            markdown_lines.append("### 📋 상황 설명")
            markdown_lines.append("")
            for participant, description in case['participants'].items():
                markdown_lines.append(f"- **({participant})** {description}")
            markdown_lines.append("")
        
        # 기본 과실비율
        if case['basic_fault_ratio'] or case['fault_details']:
            markdown_lines.append("### ⚖️ 기본 과실비율")
            markdown_lines.append("")
            
            if case['fault_details']:
                for participant, ratio in case['fault_details'].items():
                    markdown_lines.append(f"- **{participant}**: {ratio}%")
            elif case['basic_fault_ratio']:
                markdown_lines.append(f"- {case['basic_fault_ratio']}")
            
            markdown_lines.append("")
        
        # 조정 요소 표
        if case['adjustments'] or case['special_conditions']:
            markdown_lines.append("### 📊 과실비율 조정요소")
            markdown_lines.append("")
            markdown_lines.append("| 구분 | 조정 내용 | 조정값 |")
            markdown_lines.append("|------|-----------|--------|")
            
            # 일반 조정 요소
            for adj in case['adjustments']:
                number = adj['number'] if adj['number'] else ''
                description = adj['description']
                value = adj['value']
                markdown_lines.append(f"| {number} | {description} | {value} |")
            
            # 특수 조건 (비적용 등)
            for condition in case['special_conditions']:
                desc = condition['description']
                ctype = condition['type']
                markdown_lines.append(f"| | {desc} | {ctype} |")
            
            markdown_lines.append("")
        
        # 관련 법령
        if 'related_laws' in case and case['related_laws']:
            markdown_lines.append("### 📜 관련 법령")
            markdown_lines.append("")
            for law in case['related_laws']:
                markdown_lines.append(f"- {law}")
            markdown_lines.append("")
        
        # 구분선
        markdown_lines.append("---")
        markdown_lines.append("")
    
    return '\n'.join(markdown_lines)

def process_pdf_to_table_markdown(pdf_path):
    """PDF를 표 구조 마크다운으로 변환"""
    
    result = extract_and_structure_from_pdf(pdf_path)
    if not result:
        return None
    
    structured_markdown, cases = result
    
    # 출력 파일 경로
    base_name = os.path.splitext(os.path.basename(pdf_path))[0]
    output_path = os.path.join(os.path.dirname(pdf_path), f"{base_name}_표정제.md")
    
    # 파일 저장
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(structured_markdown)
    
    print(f"  ✅ 표 정제 완료: {output_path}")
    print(f"  📄 생성된 문서 크기: {len(structured_markdown):,} 문자")
    
    return structured_markdown, output_path, cases

def preview_table_markdown(content, num_lines=50):
    """생성된 표 마크다운 미리보기"""
    
    lines = content.split('\n')
    print(f"\n📋 표 정제 결과 미리보기 (처음 {num_lines}줄):")
    print("=" * 70)
    
    for i, line in enumerate(lines[:num_lines]):
        print(f"{i+1:3d}: {line}")
    
    if len(lines) > num_lines:
        print(f"... (총 {len(lines):,}줄 중 {num_lines}줄만 표시)")

def main():
    """메인 실행 함수"""
    
    # PDF 파일 경로
    pdf_files = [
        r"C:\project\2stProject_jun\jun\과실비율PDF\과실비율 원본 PDF\231107_과실비율인정기준_온라인용.pdf",
        r"C:\project\2stProject_jun\jun\과실비율PDF\과실비율 원본 PDF\(최종)과실비율심의사례_(54MB).pdf"
    ]
    
    results = []
    
    print("🚀 PDF → 표 구조 마크다운 변환 시작")
    print("=" * 70)
    
    for pdf_path in pdf_files:
        print(f"\n📂 처리 대상: {os.path.basename(pdf_path)}")
        
        result = process_pdf_to_table_markdown(pdf_path)
        if result:
            content, output_path, cases = result
            results.append({
                'pdf_path': pdf_path,
                'output_path': output_path,
                'content': content,
                'cases': cases,
                'filename': os.path.basename(output_path)
            })
            
            # 미리보기
            preview_table_markdown(content)
            print("\n" + "="*70)
    
    # 전체 결과 요약
    print("\n📊 전체 변환 결과 요약")
    print("=" * 70)
    
    total_cases = 0
    total_chars = 0
    
    for i, result in enumerate(results):
        case_count = len(result['cases'])
        char_count = len(result['content'])
        total_cases += case_count
        total_chars += char_count
        
        print(f"{i+1}. {result['filename']}")
        print(f"   📄 추출된 사례: {case_count}개")
        print(f"   📝 문서 크기: {char_count:,} 문자")
        print(f"   💾 저장 위치: {result['output_path']}")
        print()
    
    print(f"📈 총 사례 수: {total_cases}개")
    print(f"📄 총 문서 크기: {total_chars:,} 문자")
    print(f"📁 생성된 파일: {len(results)}개")
    
    print("\n🎯 다음 단계:")
    print("1. 생성된 표 구조 마크다운 확인")
    print("2. 필요시 조정 및 수정")
    print("3. DOCS 통합 변환 또는 RAG 데이터셋 생성")
    
    return results

# 실행
if __name__ == "__main__":
    results = main()

🚀 PDF → 표 구조 마크다운 변환 시작

📂 처리 대상: 231107_과실비율인정기준_온라인용.pdf
📄 PDF 처리 시작: 231107_과실비율인정기준_온라인용.pdf
  📄 페이지 1/600 추출 완료
  📄 페이지 2/600 추출 완료
  📄 페이지 3/600 추출 완료
  📄 페이지 4/600 추출 완료
  📄 페이지 5/600 추출 완료
  📄 페이지 6/600 추출 완료
  📄 페이지 7/600 추출 완료
  📄 페이지 8/600 추출 완료
  📄 페이지 9/600 추출 완료
  📄 페이지 10/600 추출 완료
  📄 페이지 11/600 추출 완료
  📄 페이지 12/600 추출 완료
  📄 페이지 13/600 추출 완료
  📄 페이지 14/600 추출 완료
  📄 페이지 15/600 추출 완료
  📄 페이지 16/600 추출 완료
  📄 페이지 17/600 추출 완료
  📄 페이지 18/600 추출 완료
  📄 페이지 19/600 추출 완료
  📄 페이지 20/600 추출 완료
  📄 페이지 21/600 추출 완료
  📄 페이지 22/600 추출 완료
  📄 페이지 23/600 추출 완료
  📄 페이지 24/600 추출 완료
  📄 페이지 25/600 추출 완료
  📄 페이지 26/600 추출 완료
  📄 페이지 27/600 추출 완료
  📄 페이지 28/600 추출 완료
  📄 페이지 29/600 추출 완료
  📄 페이지 30/600 추출 완료
  📄 페이지 31/600 추출 완료
  📄 페이지 32/600 추출 완료
  📄 페이지 33/600 추출 완료
  📄 페이지 34/600 추출 완료
  📄 페이지 35/600 추출 완료
  📄 페이지 36/600 추출 완료
  📄 페이지 37/600 추출 완료
  📄 페이지 38/600 추출 완료
  📄 페이지 39/600 추출 완료
  📄 페이지 40/600 추출 완료
  📄 페이지 41/600 추출 완료
  📄 페이지 42/600 추출 완료
  📄 페이지 43/600 추출 완료
  📄 페이지 4

  📄 페이지 384/600 추출 완료
  📄 페이지 385/600 추출 완료
  📄 페이지 386/600 추출 완료
  📄 페이지 387/600 추출 완료
  📄 페이지 388/600 추출 완료
  📄 페이지 389/600 추출 완료
  📄 페이지 390/600 추출 완료
  📄 페이지 391/600 추출 완료
  📄 페이지 392/600 추출 완료
  📄 페이지 393/600 추출 완료
  📄 페이지 394/600 추출 완료
  📄 페이지 395/600 추출 완료
  📄 페이지 396/600 추출 완료
  📄 페이지 397/600 추출 완료
  📄 페이지 398/600 추출 완료
  📄 페이지 399/600 추출 완료
  📄 페이지 400/600 추출 완료
  📄 페이지 401/600 추출 완료
  📄 페이지 402/600 추출 완료
  📄 페이지 403/600 추출 완료
  📄 페이지 404/600 추출 완료
  📄 페이지 405/600 추출 완료
  📄 페이지 406/600 추출 완료
  📄 페이지 407/600 추출 완료
  📄 페이지 408/600 추출 완료
  📄 페이지 409/600 추출 완료
  📄 페이지 410/600 추출 완료
  📄 페이지 411/600 추출 완료
  📄 페이지 412/600 추출 완료
  📄 페이지 413/600 추출 완료
  📄 페이지 414/600 추출 완료
  📄 페이지 415/600 추출 완료
  📄 페이지 416/600 추출 완료
  📄 페이지 417/600 추출 완료
  📄 페이지 418/600 추출 완료
  📄 페이지 419/600 추출 완료
  📄 페이지 420/600 추출 완료
  📄 페이지 421/600 추출 완료
  📄 페이지 422/600 추출 완료
  📄 페이지 423/600 추출 완료
  📄 페이지 424/600 추출 완료
  📄 페이지 425/600 추출 완료
  📄 페이지 426/600 추출 완료
  📄 페이지 427/600 추출 완료
  📄 페이지 428/600 추출 완료
  📄 페이지 42

  📄 페이지 125/472 추출 완료
  📄 페이지 126/472 추출 완료
  📄 페이지 127/472 추출 완료
  📄 페이지 128/472 추출 완료
  📄 페이지 129/472 추출 완료
  📄 페이지 130/472 추출 완료
  📄 페이지 131/472 추출 완료
  📄 페이지 132/472 추출 완료
  📄 페이지 133/472 추출 완료
  📄 페이지 134/472 추출 완료
  📄 페이지 135/472 추출 완료
  📄 페이지 136/472 추출 완료
  📄 페이지 137/472 추출 완료
  📄 페이지 138/472 추출 완료
  📄 페이지 139/472 추출 완료
  📄 페이지 140/472 추출 완료
  📄 페이지 141/472 추출 완료
  📄 페이지 142/472 추출 완료
  📄 페이지 143/472 추출 완료
  📄 페이지 144/472 추출 완료
  📄 페이지 145/472 추출 완료
  📄 페이지 146/472 추출 완료
  📄 페이지 147/472 추출 완료
  📄 페이지 148/472 추출 완료
  📄 페이지 149/472 추출 완료
  📄 페이지 150/472 추출 완료
  📄 페이지 151/472 추출 완료
  📄 페이지 152/472 추출 완료
  📄 페이지 153/472 추출 완료
  📄 페이지 154/472 추출 완료
  📄 페이지 155/472 추출 완료
  📄 페이지 156/472 추출 완료
  📄 페이지 157/472 추출 완료
  📄 페이지 158/472 추출 완료
  📄 페이지 159/472 추출 완료
  📄 페이지 160/472 추출 완료
  📄 페이지 161/472 추출 완료
  📄 페이지 162/472 추출 완료
  📄 페이지 163/472 추출 완료
  📄 페이지 164/472 추출 완료
  📄 페이지 165/472 추출 완료
  📄 페이지 166/472 추출 완료
  📄 페이지 167/472 추출 완료
  📄 페이지 168/472 추출 완료
  📄 페이지 169/472 추출 완료
  📄 페이지 17

In [ ]:
import re
import os

class KoreanMarkdownCleaner:
    def __init__(self):
        # 정제 통계를 위한 카운터
        self.stats = {
            'lines_processed': 0,
            'numbering_fixed': 0,
            'empty_lines_removed': 0,
            'whitespace_normalized': 0,
            'korean_numbering_detected': 0
        }
        
        # 한국어 번호 체계 패턴 정의
        self.korean_patterns = {
            # 가, 나, 다 패턴
            'korean_letters': r'^\s*([가-힣])\s*[\.\)]\s*(.*)',
            # 1, 2, 3 패턴
            'arabic_numbers': r'^\s*(\d+)\s*[\.\)]\s*(.*)',
            # ①, ②, ③ 둥근 숫자 패턴
            'circled_numbers': r'^\s*([①-⑳㉑-㉟])\s*(.*)',
            # ㄱ, ㄴ, ㄷ 자음 패턴
            'consonants': r'^\s*([ㄱ-ㅎ])\s*[\.\)]\s*(.*)',
            # (1), (2), (3) 괄호 숫자 패턴
            'parenthesis_numbers': r'^\s*\((\d+)\)\s*(.*)',
            # 가), 나), 다) 패턴
            'korean_parenthesis': r'^\s*([가-힣])\)\s*(.*)',
            # 1), 2), 3) 패턴
            'number_parenthesis': r'^\s*(\d+)\)\s*(.*)'
        }
    
    def clean_markdown_file(self, input_path, output_path=None):
        """
        마크다운 파일을 읽어서 정제하고 저장
        """
        try:
            with open(input_path, 'r', encoding='utf-8') as file:
                content = file.read()
        except UnicodeDecodeError:
            # UTF-8로 읽기 실패 시 다른 인코딩 시도
            with open(input_path, 'r', encoding='cp949') as file:
                content = file.read()
        
        cleaned_content = self.clean_markdown_text(content)
        
        # 출력 경로가 없으면 원본 파일명에 _cleaned 추가
        if output_path is None:
            base, ext = os.path.splitext(input_path)
            output_path = f"{base}_cleaned{ext}"
        
        with open(output_path, 'w', encoding='utf-8') as file:
            file.write(cleaned_content)
        
        return output_path, self.stats
    
    def clean_markdown_text(self, text):
        """
        마크다운 텍스트를 정제하는 메인 함수
        """
        lines = text.split('\n')
        cleaned_lines = []
        
        for i, line in enumerate(lines):
            self.stats['lines_processed'] += 1
            
            # 1. 기본 공백 정리
            cleaned_line = self._normalize_whitespace(line)
            
            # 2. 빈 줄 처리
            if not cleaned_line.strip():
                if i > 0 and cleaned_lines and cleaned_lines[-1].strip():
                    cleaned_lines.append('')
                continue
            
            # 3. 한국어 번호 체계 정제
            cleaned_line = self._process_korean_numbering(cleaned_line)
            
            cleaned_lines.append(cleaned_line)
        
        # 4. 연속된 빈 줄 제거
        result = self._remove_excessive_empty_lines(cleaned_lines)
        
        # 5. 최종 구조 정리
        result = self._finalize_structure(result)
        
        return '\n'.join(result)
    
    def _normalize_whitespace(self, line):
        """
        기본적인 공백 정규화
        """
        # 탭을 스페이스로 변환
        line = line.replace('\t', '    ')
        # 연속된 스페이스를 단일 스페이스로
        line = re.sub(r' {2,}', ' ', line)
        # 줄 끝 공백 제거
        return line.rstrip()
    
    def _process_korean_numbering(self, line):
        """
        한국어 번호 체계를 마크다운 리스트로 변환
        """
        original_line = line
        
        # 각 패턴을 순서대로 확인
        for pattern_name, pattern in self.korean_patterns.items():
            match = re.match(pattern, line)
            if match:
                self.stats['korean_numbering_detected'] += 1
                self.stats['numbering_fixed'] += 1
                
                if pattern_name == 'korean_letters':
                    # 가, 나, 다 -> 1. 가항목, 2. 나항목
                    letter, content = match.groups()
                    order = ord(letter) - ord('가') + 1
                    return f"{order}. **{letter}항목** {content.strip()}"
                
                elif pattern_name == 'arabic_numbers':
                    # 1, 2, 3 -> 표준 마크다운 리스트
                    number, content = match.groups()
                    return f"{number}. {content.strip()}"
                
                elif pattern_name == 'circled_numbers':
                    # ①, ②, ③ -> (1), (2), (3)
                    circled, content = match.groups()
                    number = self._circled_to_number(circled)
                    return f"   ({number}) {content.strip()}"
                
                elif pattern_name == 'consonants':
                    # ㄱ, ㄴ, ㄷ -> - ㄱ항목, - ㄴ항목
                    consonant, content = match.groups()
                    return f"   - **{consonant}항목** {content.strip()}"
                
                elif pattern_name == 'parenthesis_numbers':
                    # (1), (2), (3) -> 서브리스트
                    number, content = match.groups()
                    return f"   ({number}) {content.strip()}"
                
                elif pattern_name == 'korean_parenthesis':
                    # 가), 나), 다) -> 서브리스트
                    letter, content = match.groups()
                    return f"   - **{letter}항목** {content.strip()}"
                
                elif pattern_name == 'number_parenthesis':
                    # 1), 2), 3) -> 서브리스트
                    number, content = match.groups()
                    return f"   - {content.strip()}"
        
        return line
    
    def _circled_to_number(self, circled_char):
        """
        둥근 숫자를 일반 숫자로 변환
        """
        circled_map = {
            '①': 1, '②': 2, '③': 3, '④': 4, '⑤': 5,
            '⑥': 6, '⑦': 7, '⑧': 8, '⑨': 9, '⑩': 10,
            '⑪': 11, '⑫': 12, '⑬': 13, '⑭': 14, '⑮': 15,
            '⑯': 16, '⑰': 17, '⑱': 18, '⑲': 19, '⑳': 20,
            '㉑': 21, '㉒': 22, '㉓': 23, '㉔': 24, '㉕': 25,
            '㉖': 26, '㉗': 27, '㉘': 28, '㉙': 29, '㉚': 30,
            '㉛': 31, '㉜': 32, '㉝': 33, '㉞': 34, '㉟': 35
        }
        return circled_map.get(circled_char, 1)
    
    def _remove_excessive_empty_lines(self, lines):
        """
        연속된 빈 줄을 단일 빈 줄로 변환
        """
        result = []
        prev_empty = False
        
        for line in lines:
            if not line.strip():
                if not prev_empty:
                    result.append('')
                    prev_empty = True
                else:
                    self.stats['empty_lines_removed'] += 1
            else:
                result.append(line)
                prev_empty = False
        
        return result
    
    def _finalize_structure(self, lines):
        """
        최종 구조 정리
        """
        result = []
        
        for i, line in enumerate(lines):
            # 리스트 항목 간 적절한 간격 추가
            if i > 0 and lines[i-1].strip():
                # 메인 항목 앞에 빈 줄 추가
                if re.match(r'^\d+\.\s', line) and not re.match(r'^\d+\.\s', lines[i-1]):
                    result.append('')
            
            result.append(line)
        
        return result
    
    def print_stats(self):
        """
        정제 통계 출력
        """
        print("=== 정제 통계 ===")
        print(f"처리된 줄 수: {self.stats['lines_processed']}")
        print(f"한국어 번호 체계 감지: {self.stats['korean_numbering_detected']}")
        print(f"번호 체계 수정: {self.stats['numbering_fixed']}")
        print(f"제거된 빈 줄: {self.stats['empty_lines_removed']}")

# 사용 예제
def main():
    cleaner = KoreanMarkdownCleaner()
    
    # 샘플 텍스트로 테스트
    sample_text = """
가. 첫 번째 항목입니다
    ① 세부 항목 1
    ② 세부 항목 2
    
나. 두 번째 항목입니다
    ㄱ. 하위 항목 가
    ㄴ. 하위 항목 나
    
1 세 번째 항목
(1) 괄호 항목 1
(2) 괄호 항목 2

다) 네 번째 항목
1) 번호 항목
2) 번호 항목 2
"""
    
    print("원본 텍스트:")
    print(sample_text)
    print("\n" + "="*50 + "\n")
    
    cleaned = cleaner.clean_markdown_text(sample_text)
    print("정제된 텍스트:")
    print(cleaned)
    
    cleaner.print_stats()

if __name__ == "__main__":
    # 파일 처리 예제
    # output_path, stats = cleaner.clean_markdown_file("input.md", "output.md")
    
    # 샘플 테스트 실행
    main()